### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
learning_rate = 0.0001
batch_size = 1
N = 256
max_pool = 256
regularization = 0.0001

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "pretraining"
config_model["smooth_size"] = 10
idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DAS_train_front"

In [5]:
from models.adapt import Adapt

full_id = 'flat-limit-0477'+idd
config_model["alpha"] = 0.1

folder='DAS_train_front'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)
model.create_saver()
model.restore_model('pretraining', full_id)

ID : AdaptiveNet-noisy-cake-1674
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/pretraining/AdaptiveNet-flat-limit-0477-N=256--alpha=0.0001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.0001--smooth_size=10--type=pretraining-/model.ckpt


## Connect DPCL model to the front end

In [6]:
from models.das import DAS

with model.graph.as_default():
    model.connect_front(DAS)
    model.sepNet.output = model.sepNet.prediction
    model.cost = model.sepNet.cost
    model.freeze_front()
    model.optimize
    model.tensorboard_init()

<utils.ops.ops.Conv2D instance at 0x7f09bc228950>
<utils.ops.ops.BatchNorm instance at 0x7f09bc228830>
<utils.ops.ops.Activation instance at 0x7f09bc228b90>
<utils.ops.ops.Dropout instance at 0x7f09bc228a28>
<utils.ops.ops.Conv2D instance at 0x7f09bc2289e0>
<utils.ops.ops.BatchNorm instance at 0x7f09bc228998>
<utils.ops.ops.Activation instance at 0x7f09bc228bd8>
<utils.ops.ops.Conv2D instance at 0x7f09bc228c20>
<utils.ops.ops.BatchNorm instance at 0x7f09bc228c68>
<utils.ops.ops.Activation instance at 0x7f09bc228cb0>
<utils.ops.ops.Dropout instance at 0x7f09bc228cf8>
<utils.ops.ops.Conv2D instance at 0x7f09bc228d40>
<utils.ops.ops.BatchNorm instance at 0x7f09bc228d88>
<utils.ops.ops.Activation instance at 0x7f09bc228dd0>
<utils.ops.ops.Conv2D instance at 0x7f09bc228e18>
<utils.ops.ops.BatchNorm instance at 0x7f09bc228e60>
<utils.ops.ops.Activation instance at 0x7f09bc228ea8>
<utils.ops.ops.Dropout instance at 0x7f09bc228ef0>
<utils.ops.ops.Conv2D instance at 0x7f09bc228f38>
<utils.ops.o

In [7]:
# with model.graph.as_default():
#     uninit =[v for v in tf.global_variables() if v.name.split(':')[0] in set(model.sess.run(tf.report_uninitialized_variables()))]
#     print uninit
#     init = tf.variables_initializer(uninit)

from itertools import compress
with model.graph.as_default():
    global_vars = tf.global_variables()
    is_not_initialized = model.sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))
    print not_initialized_vars
    if len(not_initialized_vars):
        init = tf.variables_initializer(not_initialized_vars)


[<tf.Variable 'centroids:0' shape=(40, 40) dtype=float32_ref>, <tf.Variable 'prediction/W:0' shape=(128, 40) dtype=float32_ref>, <tf.Variable 'prediction/Variable:0' shape=(40,) dtype=float32_ref>, <tf.Variable 'prediction/conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>, <tf.Variable 'prediction/conv2d/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/moving_mean:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/moving_variance:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/conv2d_1/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'prediction/conv2d_1/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization_1/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalizatio

# Model creation

# Pretraining the model 

In [8]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    print Ind
    c = model.train(X_mix, X_in,learning_rate, i, ind_train=Ind)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

[[33  5]]
Step # 0  loss= 0.693147
DAS model saved at iteration number  0  with cost =  0.693147
[[21  1]]
Step # 1  loss= 0.693232
[[16 38]]
Step # 2  loss= 0.693213
[[34 19]]
Step # 3  loss= 0.693072
[[12  7]]
Step # 4  loss= 0.693011
[[25  6]]
Step # 5  loss= 0.693301
[[15  2]]
Step # 6  loss= 0.693258
[[ 0 39]]
Step # 7  loss= 0.693677
[[13 10]]
Step # 8  loss= 0.693133
[[15 39]]
Step # 9  loss= 0.693237
[[ 0 19]]
Step # 10  loss= 0.693534
[[36  7]]
Step # 11  loss= 0.693429
[[15 35]]
Step # 12  loss= 0.693147
[[16 35]]
Step # 13  loss= 0.693139
[[25 10]]
Step # 14  loss= 0.693286
[[18 10]]
Step # 15  loss= 0.693202
[[25 35]]
Step # 16  loss= 0.693118
[[37 38]]
Step # 17  loss= 0.693158
[[28  6]]
Step # 18  loss= 0.69313
[[28  6]]
Step # 19  loss= 0.693299
[[21 39]]
Step # 20  loss= 0.693552
DAS model saved at iteration number  20  with cost =  0.693552
[[36 30]]
Step # 21  loss= 0.693117
[[21  1]]
Step # 22  loss= 0.693073
[[18  5]]
Step # 23  loss= 0.692833
[[ 3 39]]
Step # 24  l

KeyboardInterrupt: 